In [ ]:
!pip install catboost xgboost lightgbm --upgrade

# Семинар градиентный бустинг



In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
np.random.seed(42)
X = np.linspace(0,10,300).reshape(-1,1)
y = np.sinc(X).ravel()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Data points')
plt.title('Generated Regression Dataset')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
# Решим задачу решающим деревом глубины 3
tree = DecisionTreeRegressor(max_depth=3, random_state=42)
tree.fit(X, y)

y_pred_tree = tree.predict(X)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Data points')
plt.plot(X, y_pred_tree, color='red', label='Decision Tree Prediction', linewidth=2)
plt.title('Prediction with a Single Decision Tree')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
# Добавим второй шаг, посчитаем разницу между метками и получившимся предсказанием
residuals = y - y_pred_tree

# Построим дерево решений, которое будем предсказывать residuals
tree_residual = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_residual.fit(X, residuals)

y_pred_residual = tree_residual.predict(X)

In [ ]:
# Суммируем предсказания двух деревьев решений
y_pred_boosted = y_pred_tree + y_pred_residual

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Data points')
plt.plot(X, y_pred_boosted, color='green', label='Boosted Prediction', linewidth=2)
plt.title('Prediction with One Step of Gradient Boosting')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gbr.fit(X, y)

y_pred_gbr = gbr.predict(X)

plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Data points')
plt.plot(X, y_pred_gbr, color='yellow', label='Gradient Boosting Prediction', linewidth=3)
plt.title('Prediction with Gradient Boosting')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
mse_tree = mean_squared_error(y, y_pred_tree)
mse_boosted = mean_squared_error(y, y_pred_boosted)
mse_gbr = mean_squared_error(y, y_pred_gbr)

mse_tree, mse_boosted, mse_gbr

## Градиентный Бустинг

Основная идея заключается в последовательном построении моделей, каждая из которых пытается исправить ошибки предыдущей:

1. **Инициализация**:
   - Начинаем с базовой модели $F_0(x)$, которая делает начальные предсказания.

2. **Вычисление Остатков**:
   - Рассчитываем остатки $ r_i $ (разницу между фактическими значениями $ y_i $ и предсказаниями текущей модели $ F_m(x_i) $):
   $$
   r_i = y_i - F_m(x_i)
   $$

3. **Обучение Новой Модели**:
   - Обучаем новую модель $ h_m(x) $ (обычно дерево решений) предсказывать эти остатки.

4. **Обновление Предсказаний**:
   - Обновляем предсказания, добавляя предсказания новой модели $ h_m(x) $ к предсказаниям предыдущей модели $ F_m(x) $:
   $$
   F_{m+1}(x) = F_m(x) + \nu \cdot h_m(x)
   $$
   где $ \nu $ — коэффициент обучения (learning rate).

5. **Повторение**:
   - Повторяем шаги 2-4 заданное количество $ M $ раз или до тех пор, пока модель не достигнет нужного качества.

6. **Финальная Модель**:
   - Финальная модель представляет собой сумму всех обученных моделей:
   $$
   F_M(x) = F_0(x) + \nu \sum_{m=1}^{M} h_m(x)
   $$

## Сравним случайный лес с градиентным бустингом

In [ ]:
# Выберем разное количество деревьев для обучения
n_trees_list = [10, 50, 100, 200, 300]
mse_gbr_list = []
mse_rf_list = []

In [ ]:
for n_trees in n_trees_list:
    # Gradient Boosting
    gbr = GradientBoostingRegressor(n_estimators=n_trees, max_depth=3, random_state=42)
    gbr.fit(X, y)
    y_pred_gbr = gbr.predict(X)
    mse_gbr = mean_squared_error(y, y_pred_gbr)
    mse_gbr_list.append(mse_gbr)

    # Random Forest
    rf = RandomForestRegressor(n_estimators=n_trees, max_depth=3, random_state=42)
    rf.fit(X, y)
    y_pred_rf = rf.predict(X)
    mse_rf = mean_squared_error(y, y_pred_rf)
    mse_rf_list.append(mse_rf)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(n_trees_list, mse_gbr_list, color='purple', label='Gradient Boosting MSE', marker='o')
plt.plot(n_trees_list, mse_rf_list, color='green', label='Random Forest MSE', marker='s')
plt.title('MSE vs Number of Trees')
plt.xlabel('Number of Trees')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.grid(True)
plt.show()

При небольшом числе деревьев алгоритм случайного леса работает лучше, чем градиентный бустинг. Все меняется с увеличением числа деревьев.


## Интерпретируемость признаков

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [ ]:
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gbr.fit(X_train, y_train)

In [ ]:
y_pred = gbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:
feature_importances = gbr.feature_importances_
features = X.columns

In [ ]:
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

In [ ]:
plt.figure(figsize=(12, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Gradient Boosting')
plt.gca().invert_yaxis()
plt.show()

## XGBoost

https://xgboost.readthedocs.io/en/release_3.0.0/

**Особенности**

**Построение деревьев по уровням**

Каждый уровень дерева заполняется полностью, прежде чем переходить к следующему уровню.

**Параллельные вычисления**

XGBoost поддерживает параллельные вычисления, что позволяет значительно ускорить процесс обучения. Особенно полезно при работе с большими наборами данных.

**Много оптимизаций под капотом, которые ускоряют вычисления**

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

plt.rcParams['figure.figsize'] = [20, 10]

california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели XGBoost
model = xgb.XGBRegressor(n_estimators=50, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Визуализация первого дерева
xgb.plot_tree(model, num_trees=0)

## LightGBM

https://lightgbm.readthedocs.io/en/stable/

**Особенности**

**Построение деревьев по листьям**

LightGBM использует стратегию роста по листьям. Это означает, что дерево растет, расширяя лист, который дает наибольший прирост информации. Такой подход может привести к асимметричным деревьям, но часто может достичь более высокой точности за меньшее время.

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Создание LightGBM Dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Параметры модели
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'verbose': -1
}

# Обучение модели
model = lgb.train(params, train_data, num_boost_round=100)

# Визуализация первого дерева
ax = lgb.plot_tree(model, tree_index=0, figsize=(20, 10), show_info=['split_gain', 'internal_value', 'internal_count', 'leaf_count'])

**EFB (Exclusive Feature Bundling)**

Алгоритм для улучшения эффективности обработки категориальных признаков объединяет некоторые признаки в группы, которые могут быть обработаны вместе. Это делается, чтобы снизить число вычислений.
Категориальные признаки с большим количеством уникальных значений могут быть сложными для обработки, особенно когда у категориального признака есть много уникальных значений. Категориальные признаки группируются на основе их статистических свойств или распределений.

**GOSS (Gradient-based One-Side Sampling)**

Этот метод используется для ускорения обучения за счет выборки данных на основе градиентов. GOSS сохраняет экземпляры с большими градиентами и случайно выбирает экземпляры с малыми градиентами. Это позволяет сосредоточиться на наиболее информативных данных, ускоряя обучение без значительной потери точности

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'goss',  # Использование GOSS
    'num_leaves': 31,
    'learning_rate': 0.1,
    'verbose': -1
}

# Обучение модели с GOSS
model = lgb.train(params, train_data, num_boost_round=100)

## CatBoost

https://catboost.ai/

**Особенности**

**Симметричные деревья**

CatBoost строит симметричные деревья: деревья имеют одинаковый предикат на каждом уровне.


In [ ]:
from catboost import CatBoostRegressor, Pool

In [ ]:
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pool = Pool(X_train, y_train, feature_names=list(X.columns))

model = CatBoostRegressor(
    max_depth=2, verbose=False, max_ctr_complexity=1, iterations=2).fit(pool)

model.plot_tree(
    tree_idx=0,
    pool=pool
)

**Нативная поддержка категориальных признаков**

CatBoost автоматически обрабатывает категориальные признаки, используя статистику на основе целевой переменной. Это позволяет модели эффективно работать с категориальными данными без необходимости предварительной обработки.

In [ ]:
from catboost.datasets import titanic
titanic_df = titanic()

X = titanic_df[0].drop('Survived',axis=1)
y = titanic_df[0].Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
is_cat = (X.dtypes != float)
for feature, feat_is_cat in is_cat.to_dict().items():
    if feat_is_cat:
        X[feature].fillna("NAN", inplace=True)

In [ ]:
is_cat

In [ ]:
cat_features_index = np.where(is_cat)[0]

In [ ]:
pool = Pool(X, y, cat_features=cat_features_index, feature_names=list(X.columns))

model = CatBoostRegressor(
    max_depth=2, verbose=False, max_ctr_complexity=1, iterations=2).fit(pool)

model.plot_tree(
    tree_idx=0,
    pool=pool
)


**Динамический бустинг**

Динамический бустинг — во время обучения делаются перестановки объектов для уменьшения переобучения

## Сравнение методов

In [ ]:
import time
from sklearn.datasets import fetch_california_housing, load_breast_cancer, load_digits
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier

In [ ]:
california = fetch_california_housing()
breast_cancer = load_breast_cancer()
digits = load_digits()

datasets = {
    "California Housing (Regression)": (pd.DataFrame(california.data, columns=california.feature_names), pd.Series(california.target), "regression"),
    "Breast Cancer (Classification)": (pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names), pd.Series(breast_cancer.target), "classification"),
    "Digits (Classification)": (pd.DataFrame(digits.data, columns=[f'pixel_{i}' for i in range(digits.data.shape[1])]), pd.Series(digits.target), "classification")
}

In [ ]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, task_type):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    if task_type == "regression":
        metric = mean_squared_error(y_test, y_pred)
    else:
        metric = accuracy_score(y_test, y_pred)

    return metric, train_time, predict_time

In [ ]:
# Compare models on each dataset
results = []

for name, (X, y, task_type) in datasets.items():
    print(f"\nНабор данных: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # XGBoost
    if task_type == "regression":
        xgboost_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42)
    else:
        xgboost_model = xgb.XGBClassifier(n_estimators=100, max_depth=3, random_state=42)
    mse_xgb, train_time_xgb, predict_time_xgb = evaluate_model(xgboost_model, X_train, X_test, y_train, y_test, task_type)
    print(f"  XGBoost {'MSE' if task_type == 'regression' else 'Accuracy'}: {mse_xgb}, Train Time: {train_time_xgb:.2f}s, Predict Time: {predict_time_xgb:.2f}s")

    # LightGBM
    if task_type == "regression":
        lightgbm_model = lgb.LGBMRegressor(n_estimators=100, max_depth=3, verbosity=-1, random_state=42)
    else:
        lightgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=3, verbosity=-1, random_state=42)
    mse_lgb, train_time_lgb, predict_time_lgb = evaluate_model(lightgbm_model, X_train, X_test, y_train, y_test, task_type)
    print(f"  LightGBM {'MSE' if task_type == 'regression' else 'Accuracy'}: {mse_lgb}, Train Time: {train_time_lgb:.2f}s, Predict Time: {predict_time_lgb:.2f}s")

    # CatBoost
    if task_type == "regression":
        catboost_model = CatBoostRegressor(n_estimators=100, max_depth=3, random_state=42, verbose=0)
    else:
        catboost_model = CatBoostClassifier(n_estimators=100, max_depth=3, random_state=42, verbose=0)
    mse_cb, train_time_cb, predict_time_cb = evaluate_model(catboost_model, X_train, X_test, y_train, y_test, task_type)
    print(f"  CatBoost {'MSE' if task_type == 'regression' else 'Accuracy'}: {mse_cb}, Train Time: {train_time_cb:.2f}s, Predict Time: {predict_time_cb:.2f}s")

    # GradientBoosting (scikit-learn)
    if task_type == "regression":
        sklearn_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
    else:
        sklearn_model = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
    mse_sk, train_time_sk, predict_time_sk = evaluate_model(sklearn_model, X_train, X_test, y_train, y_test, task_type)
    print(f"  GradientBoosting (sklearn) {'MSE' if task_type == 'regression' else 'Accuracy'}: {mse_sk}, Train Time: {train_time_sk:.2f}s, Predict Time: {predict_time_sk:.2f}s")

    results.append((name, "XGBoost", mse_xgb, train_time_xgb, predict_time_xgb))
    results.append((name, "LightGBM", mse_lgb, train_time_lgb, predict_time_lgb))
    results.append((name, "CatBoost", mse_cb, train_time_cb, predict_time_cb))
    results.append((name, "GradientBoosting (sklearn)", mse_sk, train_time_sk, predict_time_sk))

In [ ]:
results_df = pd.DataFrame(results, columns=["Dataset", "Model", "Metric", "Train Time", "Predict Time"])
print("\nСводная таблица результатов:")
print(results_df)

[Пример сравнения алгоритмов градиентного бустинга](https://arxiv.org/pdf/2305.17094)

## Реализация бустинга самостоятельно

In [ ]:
X, y = california.data, california.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Число деревьев в бустинге
n_trees = 100
# Learning rate
learning_rate = 0.3
# Максимальная глубина деревьев
max_depth = 3

In [ ]:
y_pred = np.mean(y_train)

In [ ]:
trees = []

# Цикл обучения градиентного бустинга
for i in range(n_trees):
    # TO DO
    # Calculate the residuals

    # Train a decision tree on the residuals

    # Update the prediction with the learning rate and the new tree's prediction

    # Store the tree

In [ ]:
def predict(X):
    # TO DO
    # Start with the initial prediction

    # Add the predictions of all trees

    pass

# Make predictions on the test set
y_pred_test = predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error: {mse}')